# Optimization for Machine Learning PA 2

This homework assignment investigates implementing some variants of Adam. We will be testing your optimizers on a simplified implementation of [GPT](https://github.com/openai/gpt-3) based on the [minGPT](https://github.com/karpathy/minGPT) repository by Andrej Karpathy. This is a model that takes as input a sequence of characters from a text file and attempts to predict the next character. This can be used to generate novel text by starting with a seed text string, and then repeatedly using the model to generate another character.

There is only one place you need to write code in this notebook, for questions **1a**, **1b**.

To turn in this homework: download as .ipynb (File -> download as .ipynb). Make the filename YOURNAME_PA2.ipynb and send via email attachment to opt4mlclass+program2@gmail.com with your name and PA2 in the subject line. Your submission should be saved with **all cells run to completion**. The final error of the best optimizer should be **less than 1.7** (and the best optimizer should be the one from question 1b).

This homework is **DUE on Monday 3/22 at 11:59 pm**.

# Tips
* You will need a GPU for this assignment. When using google colab, go to runtime->change runtime type and make sure that the type is set to GPU.

* You may decrease the number of training epochs while debugging, but please set it back to 20 and run again before submission.

* Study the provided AdaGrad implementation closely, it introduces a few pytorch functions that may be useful. You should check the documentation for these functions to see what they do.

* You may occasionally need to restart the runtime (runtime->restart runtime). Sometimes the GPUs don't release memory properly, and sometimes the progress bars get a little messed up.

* If you do not run on google colab, you may need to run the commands in the first cell in a terminal in your working directory before running the assignment.

In [ ]:
# grab the data and auxiliary code. Feel free to checkout the git repo to see
# what the model code will do.
!rm -r *
!git clone https://github.com/acutkosky/opt4mlPA2.git
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
!ls

In [ ]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [ ]:
# make deterministic
from opt4mlPA2.mingpt.utils import set_seed
set_seed(42)

In [ ]:
#import all the things
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import Optimizer

import math
from torch.utils.data import Dataset

from opt4mlPA2.mingpt.model import GPT, GPTConfig
from opt4mlPA2.mingpt.utils import sample
from opt4mlPA2.mingpt.trainer import Trainer, TrainerConfig

In [ ]:
# define a dataset class to process the textfile in pytorch.

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = list(set(data))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return math.ceil(len(self.data) / (self.block_size + 1))

    def __getitem__(self, idx):
        # we're actually going to "cheat" and pick a spot in the dataset at random
        i = np.random.randint(0, len(self.data) - (self.block_size + 1))
        chunk = self.data[i:i+self.block_size+1]
        dix = [self.stoi[s] for s in chunk]
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [ ]:
# the "block size" is the number of characters the model takes as input.
# in this case, it can look at up to 128 characters when predicting the next
# character.
block_size = 128 # spatial extent of the model for its context

In [ ]:
# you can download this file at https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
text = open('input.txt', 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

data has 1115394 characters, 65 unique.


# AdaGrad implementation
This is a simple adagrad implementation. You can also checkout the official pytorch implementation [here](https://github.com/pytorch/pytorch/blob/master/torch/optim/adagrad.py).

Take particular note of the functions `addcmul_` and `addcdiv_`. You may want to look up what they do for use in your solution.

In [ ]:
class AdaGrad(Optimizer):
  def __init__(self, params, lr=1.0, betas=(0.9,0.999), decouple=False, debias=True):
    # betas are ignored, but we keep them in the function signature so that it is the same as
    # the adam variants.
    super(AdaGrad, self).__init__(params, {'lr': lr, 'beta1': betas[0], 'beta2': betas[1], 'weight_decay': 0.0})


    for group in self.param_groups:
      for p in group['params']:
        state = self.state[p]
        state['step'] = 0
        state['v'] = torch.zeros_like(p, memory_format=torch.preserve_format)


  @torch.no_grad()
  def step(self, closure=None):
    # in this class, and also usually in practice, closure will always be None.
    loss = None
    epsilon = 1e-8
    if closure is not None:
      with torch.enable_grad():
        loss = closure()

    for group in self.param_groups:
      lr = group['lr']
      beta1 = group['beta1']
      beta2 = group['beta2']
      weight_decay = group['weight_decay']

      # it is common practice to call the model parameters p in code.
      # in class we follow more closely analytical conventions, in which the
      # parameters are often called w for weights.
      for p in group['params']:
        if p.grad is None:
          continue

        if weight_decay != 0.0:
          p.grad.add_(p, alpha=weight_decay)
        
        # Update the iteration counter (again, this is not actually used in this algorithm)
        state = self.state[p]
        state['step'] += 1


        state['v'].addcmul_(p.grad, p.grad, value=1.0)

        p.addcdiv_(p.grad, torch.sqrt(state['v']).add_(epsilon), value=-lr)

# QUESTION 1a

Implement the [AdamW](https://openreview.net/pdf?id=Bkg6RiCqY7) update *without* using the debiasing terms. AdamW performs the following (per-coordinate) update:

$$
w_{t+1} = w_t - \eta_t\left(\frac{\hat m_t}{\sqrt{\hat v_t} +\epsilon} + \lambda w_t\right)
$$
where $\hat m_t$ and $\hat v_t$ are generated the same way as in the standard [Adam](https://openreview.net/pdf?id=Bkg6RiCqY7) update, and $\lambda$ is an extra "weight decay" parameter provided to the optimizer. 

Ordinarily, "weight decay" is another word for L2 regularization. That is, the loss is modified to:
$$
\mathcal{L}(w) + \frac{\lambda}{2}\|w\|^2
$$
This means that we could implement weight decay by changing the gradient to $\nabla \mathcal{L}(w) + \lambda w$. The idea behind AdamW is that the weight-decay term is in some sense "well-understood" and should not be included in the $v_t$ and $A_t$ statistics that are being used to understand the more mysterious loss surface $\mathcal{L}(w)$. See the linked paper for more details and full pseudocode.

In your implementation, you should use the raw $m_t$ and $v_t$ values without applying the debiasing terms discussed in the papers and class.

# QUESTION 1b

Upgrade your debias-free AdamW implementation to use the `use_norm_scaling` argument of the `__init__` method. When this argument is `True`, you should scale the learning rate by the norm of the weights *for the given pytorch variable*. That is, for each variable $p$ you will replace the learning rate $\eta_t$ at time $t$ with $\|p\|\eta_t$ in the update:
$$
w_{t+1}[i] = w_t[i] - \|w_t\|_2\eta_t\left(\frac{m_t[i]}{\sqrt{v_t[i]} +\epsilon} + \lambda w_t[i]\right)
$$
When the `use_norm_scaling` argument is false, simply perform the update from question 1a.

This learning rate heuristic is inspired by a similar proposal for use with normalized updates in the [LARS](https://arxiv.org/abs/1708.03888) optimizer.

In [ ]:

class AdamW_bias(Optimizer):
  def __init__(self, params, lr=1.0, betas=(0.9,0.999), use_norm_scaling=False):
    super(AdamW_bias, self).__init__(params, {'lr': lr, 'beta1': betas[0], 'beta2': betas[1], 'weight_decay': 0.0})

    self.use_norm_scaling = use_norm_scaling

    for group in self.param_groups:
      for p in group['params']:
        ## YOUR CODE HERE ##


  @torch.no_grad()
  def step(self, closure=None):
    # in this class, and also usually in practice, closure will always be None.
    loss = None
    epsilon = 1e-8
    if closure is not None:
      with torch.enable_grad():
        loss = closure()
      
    ## YOUR CODE HERE ##



In [ ]:
# generate the configuration for the model. These parameters specify
# the neural network architecture we will be using. It is not necessary
# to understand this.
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=128)

In [ ]:
# generate training configurations for each of the optimizers. We will be testing
# adagrad
# adam (official pytorch implementation)
# adamw (official pytorch implementation)
# your optimizer both with and without the norm_scaling flag set.
def adamw_bias_factory(params, lr, betas):
  return AdamW_bias(params, lr, betas)

def adamw_bias_norm_scaling_factory(params, lr, betas):
  return AdamW_bias(params, lr, betas, use_norm_scaling=True)

optimizers = {
    'adagrad': AdaGrad, 
    'adam': torch.optim.Adam, 
    'adamw': torch.optim.AdamW, 
    'adamw_bias': adamw_bias_factory, 
    'adamw_bias_norm_scaling': adamw_bias_norm_scaling_factory
  }

training_configs = {}

for name, opt in optimizers.items():
# construct a training config: this sets the learning rate, batch size, number 
# of epochs ect for each optimizer. warmup_tokens and final_tokens are parameters
# used to setup a warm-up and decay learning rate scheduler.
  training_configs[name] = TrainerConfig(max_epochs=20, batch_size=256, learning_rate=6e-4, optimizer=opt,
                        lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset)*block_size,
                        num_workers=4)

In [ ]:
# train a model on each optimizer, keeping track of the best-performing one.
losses = {}
min_loss = float('inf')
best_model = None
best_optimizer = None
for name, tconf in training_configs.items():
  print("training new model with optimizer: {}".format(name))
  model = GPT(mconf)
  trainer = Trainer(model, train_dataset, None, tconf)
  train_loss = trainer.train()
  losses[name] = train_loss
  train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters
  print("final epoch train loss: {}".format(train_loss))
  if train_loss < min_loss:
    best_model = model
    best_optimizer = name
    min_loss = train_loss

print("best optimizer: {} with loss: {}".format(best_optimizer, min_loss))

In [ ]:
# alright, let's sample some character-level shakespear

context = "O God, O God!"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(best_model, x, 2000, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)